# Fluent Python: Chapter 7.
## Function Decorators and Closures

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [3]:
@deco
def target():
    print('running target()')

In [4]:
target()

running inner()


In [5]:
target

<function __main__.deco.<locals>.inner>

First crucial fact of decorators is they have power to replace decorated function with a different one.
Second crucial fact is they are executed immediately when the module is loaded

### registration.py Example 7-2

In [6]:
registry = []

In [7]:
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

In [8]:
@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(<function f1 at 0x10386af28>)
running register(<function f2 at 0x10387b158>)


In [9]:
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

In [10]:
main()

running main()
registry -> [<function f1 at 0x10386af28>, <function f2 at 0x10387b158>]
running f1()
running f2()
running f3()


f1 and f2 decorated functions are run right away at 'import time

#### Variable scope rules

In [11]:
def f1(a):
    print(a)
    print(b)

In [12]:
f1(3)

3


NameError: name 'b' is not defined

In [13]:
b = 6
f1(3)

3
6


In [14]:
# example 2
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [15]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When python compiles function, it knows that a local var b exists. Because it hasn't been asigned yet, b is unbound and throws an UnboundLocalError. If we want the function to use the global b definition, include global b

In [16]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [17]:
f3(3)

3
6


In [18]:
from dis import dis

In [19]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP
             20 LOAD_CONST               0 (None)
             23 RETURN_VALUE


In [20]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  5          10 LOAD_GLOBAL              0 (print)
             13 LOAD_FAST                1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  6          20 LOAD_CONST               1 (9)
             23 STORE_FAST               1 (b)
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


In [21]:
dis(f3)

  4           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  5          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  6          20 LOAD_CONST               1 (9)
             23 STORE_GLOBAL             1 (b)
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


#### Averager: class based implementation of average function that computes average on ever-increasing series of values

In [22]:
class Averager():
    
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
        

In [23]:
avg = Averager()
avg(10)

10.0

In [24]:
avg(11)

10.5

In [25]:
avg(12)

11.0

Now let's make functional implementation, make_averager, a higher-order function

In [26]:
def make_averager():
    series = [] # this is a free-variable, not bound by local scope
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [27]:
avg = make_averager()

In [28]:
avg(10)

10.0

In [29]:
avg(11)

10.5

In [30]:
avg(12)

11.0